In [1]:
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re

from konlpy.tag import Okt

Using TensorFlow backend.


# 데이터 로드

디코더 입력에 START가 들어가면 디코딩의 시작 의미. 반대로 디코더 출력에 END가 나오면 디코딩 종료

In [2]:
# 태그 단어
PAD = "<PADDING>"
STA = "<STA>"
END = "<END>"
OOV = "<OOV>"

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 1000

# 임베딩 벡터 차원
embedding_dim = 500

# LSTM 히든 레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

# 챗봇 데이터 로드
chatbot_data = pd.read_csv(r"D:/PROJECT/data/naver_worry_finally_5000.csv",encoding='cp949')
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

In [3]:
len(question)

5000

In [4]:
# 데이터의 일부만 학습에 사용
question = question[:200]
answer = answer[:200]

# 챗봇 데이터 출력
for i in range(10):
    print('Q:'+question[i])
    print('A:'+answer[i])
    print()

Q:그 친구랑 좀 다퉜는데 싸운날이 시험기간이라서 시험끝나고 얘기를 해서 풀라했는데 상대방도 동의를 했는데 결국 얘기를 못했어요.. 그래서 제가 다시 얘기좀 하자고 말했는데 그 친구가
A:시험기간이어서 시험이 끝난 이후에 대화를 하기로 서로 얘기를 나눴었나보네요. 그런데 시험이 끝나고도 친구가 대화에 응하지 않아서 그때 일을 없었던 일로 하자고 하고 그냥 끝내게 되었네요. 대화에 응하지 않는 친구의 태도가 비공개님에게는 상처가 된 것 같아요. 서로 오해가 있어서 다툼이 있었다면 그 친구도 오해를 풀고 싶은 마음이 있을거에요. 약속한 날짜에 정말 다른 일이 있어서 약속을 지키지 못했을거라고 생각해봐요. 물론 비공개님과의 약속이 먼저였어야 하는데 그렇지 못했던 것은 서운했을 수 있어요. 그 친구에게 다시 한번 그때의 일에 대해서 대화로 풀어보자고 제안을 해보는것이 좋겠어요. 그렇게 해야 서로 마음이 편해질 수 있을 것 같아요.

Q:전에는 초경이 하고싶었는데 증상(?)같은데 요즘 나타나서 걱정이 되네요ㅜ 겨털이랑 음모났구요 가슴에 몽우리 잡힌지는 1~2년 정도 좀 오래됐어요 냉도 나온지 좀 됐구요 요즘에는 배
A:먼저 초경을 시작하는 적정한 나이는  만 11세에서 만 13세로 보고 있습니다. 키가 많이 자라지 않거나 2차 성징이 없다면 만 13세까지 키가 많이 자라면서 2차 성징이 있다면  16세까지도 기다릴 수 있다고 보는데요. 먼저, 초경 전 증상으로는 가슴 몽우리가 생기고,  가슴이 봉긋하게 올라온답니다. 개인차가 있지만 보통 여자 청소년들은 가슴에 몽우리가 생기고 나서 1년 6개월에서 2년 정도 후에 초경을 시작하게 된다고 합니다. 또한. 음모가 나기 시작하고, 냉 분비물이 많아지게 되고요. 일반적으로 노란색이나 흰색의 분비물(냉)이 묻어나오지만 초경을 시작할 때가 되면 분비물의 색이 갈색으로 바뀌게 되는데, 이렇게 갈색의 분비물이 묻어나오게 되면  보통 몇 달 안에 초경을 경험하게 된다고 해요. 그러나 사람마다 개인차가 있어 갈색 분비물이 나오고도  오랜

# 단어 사전 생성

In [5]:
# 형태소 분석 함수 
def pos_tag(sentences):
    # KoNLPy 형태소 분석기 설정
    tagger = Okt()
    # 문장 품사 변수 초기화
    sentences_pos = []
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER,"",sentence)

        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        # 형태소 단위로 문자열을 끊고 싶다면, .morphs()를 사용하면 된다
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
    return sentences_pos

In [6]:
# 형태소 분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소 분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q:' + question[i])
    print('A:' + answer[i])
    print()

Q:그 친구 랑 좀 다퉜는데 싸운 날 이 시험 기간 이라서 시험 끝나고 얘기 를 해서 풀라 했는데 상대방 도 동의 를 했는데 결국 얘기 를 못 했어요 그래서 제 가 다시 얘기 좀 하자 고 말 했는데 그 친구 가
A:시험 기간 이어서 시험 이 끝난 이후 에 대화 를 하기로 서로 얘기 를 나눴었나 보네요 그런데 시험 이 끝나고도 친구 가 대화 에 응 하지 않아서 그때 일 을 없었던 일로 하자 고 하고 그냥 끝내게 되었네요 대화 에 응 하지 않는 친구 의 태도 가 비공개 님 에게는 상처 가 된 것 같아요 서로 오해 가 있어서 다툼 이 있었다면 그 친구 도 오해 를 풀 고 싶은 마음 이 있을거에요 약속 한 날짜 에 정말 다른 일이 있어서 약속 을 지키지 못 했을거라고 생각 해봐요 물론 비공개 님 과의 약속 이 먼저 였어야 하는데 그렇지 못 했던 것 은 서운했을 수 있어요 그 친구 에게 다시 한번 그때 의 일 에 대해 서 대화 로 풀어 보자고 제안 을 해보는것이 좋겠어요 그렇게 해야 서로 마음 이 편해질 수 있을 것 같아요

Q:전 에는 초경 이 하고싶었는데 증상 같은데 요즘 나타나서 걱정 이 되네요 ㅜ 겨털 이랑 음모 났구요 가슴 에 몽 우리 잡힌지는 12년 정도 좀 오래 됐어요 냉 도 나온지 좀 됐구요 요즘 에는 배
A:먼저 초경 을 시작 하는 적정한 나이 는 만 11 세 에서 만 13 세로 보고 있습니다 키 가 많이 자라지 않거나 2 차 성 징 이 없다면 만 13 세 까지 키 가 많이 자라면서 2 차 성 징 이 있다면 16 세 까지도 기다릴 수 있다고 보는데요 먼저 초경 전 증상 으로는 가슴 몽우리 가 생기 고 가슴 이 봉 긋하게 올라온답니다 개인 차가 있지만 보통 여자 청소년 들 은 가슴 에 몽우리 가 생기 고 나서 1년 6 개월 에서 2년 정도 후 에 초경 을 시작 하게 된다고 합니다 또한 음모 가 나기 시작 하고 냉 분비물 이 많아지게 되고요 일반 적 으로 노란색 이나 흰색 의 분비물 냉이 묻어나오지만 초경 을 시작 할 때 가 되면 분비물 의 색 이 갈색 으

In [7]:
# 질문과 대답 문장들을 하나로 합침
sentences=[]
sentences.extend(question)
sentences.extend(answer)

words=[]

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)
# 길이가 0인 단어는 삭제
words = [word for word in words if len(word)>0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

질문과 대답 문장들을 합쳐서 전체 단어사전 만들기. 자연어 처리에서는 항상 이렇게 단어를 인덱스에 따라서 정리.그래야지 문장을 인덱스 배열로 바꿔서 임베딩 레이어에 넣을 수 있습니다. 
또한 모델의 출력에서 나온 인덱스를 다시 단어로 변환하는데도 필요

In [8]:
# 단어 개수
len(words)

6074

In [9]:
# 단어 출력
words

['<PADDING>',
 '<STA>',
 '<END>',
 '<OOV>',
 '기',
 '정신',
 '전화해',
 '언니',
 '가지면서',
 '그런데도',
 '잠도',
 '조율',
 '필요하면',
 '깨지고',
 '딴에는',
 '소심해요',
 '링크',
 '미치는',
 '큰데',
 '질',
 '도제',
 '불편함을',
 '메일',
 '배우지도',
 '외',
 '차려서',
 '근원',
 '행복하겠어요',
 '29일',
 '나와서',
 '하기가',
 '입학',
 '노트',
 '기대하게',
 '질문',
 '올라와',
 '담긴',
 '나오게',
 '두지',
 '와의',
 '구입',
 '들어주세요',
 '강한',
 '1620분',
 '아니다라고',
 '나가는',
 '기분',
 '조차도',
 '그리운',
 '4년',
 '숙식',
 '실패',
 '한',
 '함께',
 '얘',
 '에게는',
 '바뀔수',
 '됬습니다',
 '종일',
 '때문',
 '취직',
 '인지',
 '오래',
 '감고',
 '조조',
 '가지는',
 '해소',
 '친밀',
 '눌러졌던',
 '귀엽고',
 '기운',
 '온통',
 '만날',
 '드네',
 '크고',
 '무력감',
 '언론',
 '적용',
 '일상생활',
 '찾아보는건',
 '열',
 '인데용',
 '듯',
 '있을까',
 '래',
 '살',
 '움직입니다',
 '선택',
 '궁금하네요',
 '앉아있는',
 '낫고요',
 '구걸',
 '예측',
 '저려서',
 '되었으면',
 '퍼센트',
 '커졌네요',
 '지식',
 '두면',
 '자란',
 '처방전',
 'wwwacademyinfogokr',
 '틈',
 '기억',
 '불안하다',
 '보려는',
 '나지',
 '신고',
 '골격',
 '누군가',
 '기울이는',
 '느껴졌을',
 '궁금하거나',
 'A',
 '괜찮을것',
 '72시간',
 '친',
 '삐지는것',
 '바라지',
 '합의',
 '해볼만',
 '발육',
 '가는',
 '정중한',
 '움직임',
 'a',
 '에게',

In [10]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [11]:
# 단어->인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
word_to_index

{'<PADDING>': 0,
 '<STA>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '나와': 4,
 '했지만': 5,
 '말씀': 6,
 '이동': 7,
 '간지': 8,
 '활발한': 9,
 '바뀌어야': 10,
 '입': 11,
 '봤는데': 12,
 '결정': 13,
 '170초': 14,
 '도해': 15,
 '불안정한': 16,
 '마지막': 17,
 '같은데': 18,
 '이야': 19,
 '이루어져있다는것을': 20,
 '듣거나': 21,
 '들어가면': 22,
 '그렇지': 23,
 '싶지': 24,
 '믿고': 25,
 '슬퍼하게': 26,
 '잘': 27,
 '급하다': 28,
 '퍼센트': 29,
 '꿈이기에': 30,
 '어른스럽다고': 31,
 '합의': 32,
 '당당하고': 33,
 '다른': 34,
 '넷': 35,
 '쉬어서': 36,
 '겉': 37,
 '보여요': 38,
 '주지': 39,
 '사줘도': 40,
 '30분': 41,
 '행복하겠어요': 42,
 '올라': 43,
 '불안하다면': 44,
 '쉽게': 45,
 '한마디': 46,
 '같네요': 47,
 '꾸밀수': 48,
 '나왔습니다': 49,
 '식단': 50,
 '종일': 51,
 '이예': 52,
 '께서': 53,
 '됩니다': 54,
 '중간': 55,
 '접으라는': 56,
 '늘': 57,
 '준다네요': 58,
 '실패': 59,
 '춤': 60,
 '유명하고': 61,
 '지침': 62,
 '나서': 63,
 '숨': 64,
 '여권': 65,
 '선물': 66,
 '<': 67,
 '아팠을': 68,
 '이지': 69,
 '나오게': 70,
 '써': 71,
 '읽어주시고': 72,
 '드네': 73,
 '변형': 74,
 '싶고': 75,
 '안된다면': 76,
 '현재키': 77,
 '칭찬': 78,
 '바라게': 79,
 '용기내어서': 80,
 '있다면요': 81,
 '다닐수도': 82,
 '축하': 83,
 '요인'

In [12]:
# 인덱스 -> 단어
# 문장을 인덱스로 변환하여 모델 입력으로 사용
index_to_word

{0: '<PADDING>',
 1: '<STA>',
 2: '<END>',
 3: '<OOV>',
 4: '나와',
 5: '했지만',
 6: '말씀',
 7: '이동',
 8: '간지',
 9: '활발한',
 10: '바뀌어야',
 11: '입',
 12: '봤는데',
 13: '결정',
 14: '170초',
 15: '도해',
 16: '불안정한',
 17: '마지막',
 18: '같은데',
 19: '이야',
 20: '이루어져있다는것을',
 21: '듣거나',
 22: '들어가면',
 23: '그렇지',
 24: '싶지',
 25: '믿고',
 26: '슬퍼하게',
 27: '잘',
 28: '급하다',
 29: '퍼센트',
 30: '꿈이기에',
 31: '어른스럽다고',
 32: '합의',
 33: '당당하고',
 34: '다른',
 35: '넷',
 36: '쉬어서',
 37: '겉',
 38: '보여요',
 39: '주지',
 40: '사줘도',
 41: '30분',
 42: '행복하겠어요',
 43: '올라',
 44: '불안하다면',
 45: '쉽게',
 46: '한마디',
 47: '같네요',
 48: '꾸밀수',
 49: '나왔습니다',
 50: '식단',
 51: '종일',
 52: '이예',
 53: '께서',
 54: '됩니다',
 55: '중간',
 56: '접으라는',
 57: '늘',
 58: '준다네요',
 59: '실패',
 60: '춤',
 61: '유명하고',
 62: '지침',
 63: '나서',
 64: '숨',
 65: '여권',
 66: '선물',
 67: '<',
 68: '아팠을',
 69: '이지',
 70: '나오게',
 71: '써',
 72: '읽어주시고',
 73: '드네',
 74: '변형',
 75: '싶고',
 76: '안된다면',
 77: '현재키',
 78: '칭찬',
 79: '바라게',
 80: '용기내어서',
 81: '있다면요',
 82: '다닐수도',
 83: '축하',
 84: 

# 전처리

In [11]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type):
    sentences_index = []

    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []

        # 디코더 입력일 경우 맨 앞에 START태그 추가
        if type==DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])

        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                #사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                #사전에 없는 단어면 OOV인덱스 추가
                sentence_index.extend([vocabulary[OOV]])
        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index+=[vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
        
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index+=(max_sequences-len(sentence_index))*[vocabulary[PAD]]

        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)
    

원래 seq2seq는 디코더의 현재 출력이 디코더의 다음 입력으로 들어갑니다. 다만 학습에서는 굳이 이렇게 하지 않고 디코더 입력과 디코더 출력의 데이터를 각각 만듭니다.

그러나 예측시에는 이런 방식이 불가능. 출력값을 미리 알지 못하기 때문에, 디코더 입력을 사전에 생성할 수가 없습니다. 이런 문제를 해결하기 위해 훈련 모델과 예측 모델을 따로 구성해야 합니다.

In [12]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫번째 인코더 입력 출력(12시 땡)
x_encoder[0]

array([5293,  846,  597, 2226, 1591, 3207, 2056, 3019, 5935, 2530,  480,
       5935, 1488, 5143,  600, 2202, 5917, 5307, 3709,  774, 4493,  600,
       5307, 4199, 5143,  600, 2154, 5909, 3521, 4680, 1324, 5169, 5143,
       2226, 3805, 1681, 2939, 5307, 5293,  846, 1324,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [13]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫번째 디코더 입력 출력(START 하루 가 또 가네요)
x_decoder[0]

array([   1, 5935, 2530, 1109, 5935, 3019, 3772,  496, 3560, 5414,  600,
       4639,  446, 5143,  600, 5749, 3471, 2332, 5935, 3019, 3328,  846,
       1324, 5414, 3560, 2873, 3123, 3203, 1906, 1738,  826, 6045, 1929,
       3805, 1681, 5946, 1768, 4228, 2946, 5414, 3560, 2873, 3123, 5097,
        846, 3429, 2295, 1324, 4499, 2595,   55, 3089, 1324, 1127, 2749,
       5961,  446, 3333, 1324, 2061, 3837, 3019, 1749, 5293,  846,  774,
       3333,  600, 2806, 1681, 4359,  987, 3019,  212, 3472,   52, 2375,
       3560, 2974, 2227,  818, 2061, 3472,  826, 4232, 2154, 4444, 3379,
       6067, 3813, 4499, 2595, 3930, 3472, 3019, 3389, 1380, 1811, 2382,
       2154, 5036, 2749, 4899, 4526, 2761,  475, 5293,  846,  126, 5169,
       2290, 1906, 3429, 1738, 3560, 4500,  402, 5414, 2206,  647, 4230,
        627,  826, 2899,  488, 3005, 5778,  446,  987, 3019, 1039, 2761,
       2527, 2749, 5961,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [14]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer,word_to_index, DECODER_TARGET)

# 첫 번재 디코더 목표 출력(하루 가 또 가네요 END)
y_decoder[0]
len(y_decoder)

200

In [15]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습 시 입력은 인덱스이지만, 출력은 원핫인코딩 형식
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i,j,index]=1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

인코더 입력과 디코더 입력은 임베딩 레이어에 들어가는 인덱스 배열입니다. 반면에 디코더 출력은 원핫인코딩 형식이어야 합니다. 디코더의 마지막 Dense레이어에서 Softmax로 나오기 때문

# 모델 생성

In [22]:
#-----------------------------------
# 훈련 모델 인코더 정의
#-----------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h와 state_c 2개의 상태 존재
# recurrent_dropout은 state 삭제시키는 것
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim, dropout=0.1, recurrent_dropout=0.5, return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]

#-------------------------------------
# 훈련 모델 디코더 정의
#------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words),embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)
print(decoder_outputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리가히 위함
decoder_lstm  = layers.LSTM(lstm_hidden_dim, dropout=0.1, recurrent_dropout=0.5, return_state=True, return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs,_,_=decoder_lstm(decoder_outputs, initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words),activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#-------------------------------------------
# 훈련 모델 정의
#-------------------------------------------

# 입력과 출력으로 함수형 API모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
model.summary()


Tensor("embedding_4/embedding_lookup/Identity_2:0", shape=(None, None, 500), dtype=float32)
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 500)    3037000     input_13[0][0]                   
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 500)    3037000     input_14[0][0]                   


지금까지의 예제는 Sequential방식의 모델이었습니다. 이번에는 함수형 API모델 사용. 인코더와 디코더가 따로 분리되어야 하는데, 단순히 레이어를 추가하여 붙이는 순차형으로는 구현이 불가능

Model()함수로 입력과 출력을 따로 설정하며 모델 만듭니다. 그 다음 compile과 fit은 이전과 동일하게 적용하시면 됩니다. 

In [27]:
#-----------------------------------
#예측 모델 인코더 정의
#-----------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
# encoder_states = [state_h, state_c]
encoder_model = models.Model(input=encoder_inputs, output=encoder_states)

#-----------------------------------
# 예측 모델 디코더 정의
#-----------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs, initial_state = decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs]+decoder_states_inputs, [decoder_outputs]+decoder_states)
decoder_model.summary()


Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 500)    3037000     input_14[0][0]                   
__________________________________________________________________________________________________
input_23 (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 128)          0                                            
___________________________________________________________________________________________

예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용. 예측 모델 인코더는 훈련 모델 인코더와 동일. 그러나 예측 모델 디코더는 매번 LSTM상태값을 입력 받음. 또한 디코더의 LSTM상태를 출력값과 같이 내보내서, 다음 번 입력에 넣습니다.

이렇게 하는 이유는 LSTM을 딱 한번의 타임스텝만 실행하기 때문. 그래서 매번 상태값을 새로 초기화 해야 합니다. 이와 반대로 훈련할 때는 문장 전체를 계속 LSTM으로 돌리기 때문에 자동으로 상태값이 전달됩니다. 

# 훈련 및 테스트

In [28]:
# 인덱스를 문장으롭 변환
def convert_index_to_text(indexs, vocabulary):
    sentence=''

    #모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            #종료 인덱스면 중지
            break
        elif vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            sentence+=vocabulary[OOV_INDEX]
        # 빈칸 추가
        sentence += ' '
    return sentence

In [29]:
# epoch 반복
for epoch in range(20):
    print('Total Epoch:', epoch+1)

    # 훈련 시작
    history = model.fit([x_encoder,x_decoder],
                        y_decoder, 
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도와 손실 출력
    print('accuracy:',history.history['acc'][-1])
    print('loss:',history.history['loss'][-1])

    # 문장 예측 테스트
    # (3박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[0].reshape(1, x_encoder[2].shape[0]) # input_encoder = x_encoder[2].reshape(1,130)
    input_decoder = x_decoder[0].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])

    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0],1)

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()
    results.shape()

Total Epoch: 1


In [22]:
x_encoder[2].shape

(30,)

In [23]:
input_encoder = x_encoder[2].reshape(1,130)
input_encoder.shape

(1, 30)

In [24]:
results[0]

array([[4.7223610e-12, 4.2306107e-12, 6.9863921e-15, ..., 4.4180254e-12,
        4.7222890e-12, 2.6367460e-15],
       [1.8441380e-13, 1.7150743e-13, 1.2965180e-14, ..., 1.6967558e-13,
        2.2185654e-13, 6.5482870e-23],
       [5.0385477e-12, 4.1151830e-12, 2.8387057e-14, ..., 4.0273145e-12,
        5.2463507e-12, 1.9442574e-15],
       ...,
       [5.2178322e-15, 4.7651571e-15, 5.2638629e-08, ..., 5.5106041e-15,
        5.0937931e-15, 6.7482019e-16],
       [1.3463943e-16, 1.4818873e-16, 7.4650598e-06, ..., 1.5423833e-16,
        1.6397054e-16, 4.5124483e-20],
       [2.5221050e-14, 2.3099463e-14, 9.9999881e-01, ..., 2.2189382e-14,
        2.8190854e-14, 1.5700093e-14]], dtype=float32)

학습이 진행될수록 예측 문장이 제대로 생성되는 것을 볼 수 있다. 다만 여기서의 예측은 단순히 테스트를 위한 것이라, 인코더 입력과 디코더 입력 데이터가 동시에 사용. 아래 문장 생성에서는 예측 모델을 적용하기 때문에, 오직 인코더 입력 데이터만 집어 넣습니다. 

In [25]:
# 모델 저장
encoder_model.save(r'D://PROJECT/model/seq2seq_chatbot_encoder_model.h5')
decoder_model.save(r'D://PROJECT/model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 저장
with open(r'D://PROJECT/model/word_to_index.pkl','wb')as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)
with open(r'D://PROJECT/model/index_to_word.pkl','wb')as f:
    pickle.dump(index_to_word,f,pickle.HIGHEST_PROTOCOL)


<pickle모듈><br>
일반 텍스트를 파일로 저장할 떄는 파일 입출력 이용
하지만 리스트나 클래스같은 텍스트가 아닌 자료형은 일반적인 파일 입출력 방법으로는 데이터를 저장하거나 불러올 수 없다. <br>
pickle모듈을 이용하면 원하는 데이터를 자료형의 변경 없이 파일로 저장하여 그대로 로드할 수 있다. <br>
pickle로 데이터를 저장하거나 불러올때는 파일을 바이트형식으로 읽거나 써야한다.(wb,rb)

pickle.load()는 한줄씩 데이터를 읽어오고
pickle.dump()는 뭉탱이로 읽어옴


# 문장 생성

In [26]:
# 모델 파일 로드
encoder_model = models.load_model(r'D://PROJECT//model/seq2seq_chatbot_encoder_model.h5')
decoder_model = models.load_model(r'D://PROJECT//model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 파일 로드
with open(r'D://PROJECT//model/word_to_index.pkl','rb') as f:
    word_to_index = pickle.load(f)
with open(r'D://PROJECT//model/index_to_word.pkl','rb') as f:
    index_to_word  = pickle.load(f)

In [27]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)

    return input_seq


In [28]:
# 텍스트 생성
def generate_text(input_seq):

    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    # 문장 1개, 토큰 1개
    target_seq = np.zeros((1,1))

    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0,0] = STA_INDEX

    # 인덱스 초기화
    target_seq[0,0] = STA_INDEX

    # 인덱스 초기화
    indexs = []

    # 디코더 타임 스탭 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict([target_seq]+states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0,0,:])
        indexs.append(index)

        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break
        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1,1))
        target_seq[0,0] = index

        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)

    return sentence

제일 첫 단어는 START로 시작. 그리고 출력으로 나온 인덱스를 디코더 입력으로 넣고 다시 예측 반복. 상태값을 받아 다시 입력으로 같이 넣는 것에 주의. END태그가 나오면 문장 생성 종료

In [29]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('친구랑 싸웠어요')
input_seq

array([[3241,  445,    3,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [30]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'질문 자 님 이 엄마 때문 에 꿈 을 포기 했을 때 어떻게 해야 하는지 고민 이 되어 이렇게 고민 글 을 올려 주었네요 어떤 꿈이기에 포기 할 수 '

In [31]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('친구랑 심하게 싸웠어요')
input_seq

array([[3241,  445,    3,    3,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [32]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'질문 자 님 이 엄마 때문 에 꿈 을 포기 했을 때 어떻게 해야 하는지 고민 이 되어 이렇게 고민 글 을 올려 주었네요 어떤 꿈이기에 포기 할 수 '

데이터셋 문장에서는 없던 '같이'를 단어를 추가해 보았습니다. 그래도 비슷한 의미란 것을 파악하여 동일한 답변이 나왔습니다.

In [33]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('친구랑 싸울 것 같아요')
input_seq

array([[3241,  445,    3, 1567,  885,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [34]:
sentence = generate_text(input_seq)
sentence

'질문 자 님 이 엄마 때문 에 꿈 을 포기 했을 때 어떻게 해야 하는지 고민 이 되어 이렇게 고민 글 을 올려 주었네요 어떤 꿈이기에 포기 할 수 '